--Importar librerias necesarias--

In [ ]:
#Importar librerias necesarias
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier

--Cargando del dataset desde UCI--

In [ ]:
# Cargar el conjunto de datos
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

# Definir los nombres de las columnas
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

# Cargar el conjunto de datos
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)

--Limpieza de datos--

In [ ]:
data.dropna(inplace=True)  # Eliminar filas con valores faltantes

--Separaración de features y target--

In [ ]:
X = data.drop("income", axis=1)  # Features
y = data["income"].apply(lambda x: 1 if x == ">50K" else 0)  # Target variable

--One-hot encoding para categóricas--

In [ ]:
X_encoded = pd.get_dummies(X) # One-hot encoding para categóricas

--División entre entrenamiento y testeo--

In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba
train_X, test_X, train_y, test_y = train_test_split(X_encoded, y, test_size=0.2, random_state=12)

--Entrenamiento del modelo XGBoost--

In [ ]:
start_time = time.time() # Iniciar temporizador
# Entrenar el modelo
xgb_model = XGBClassifier(
    eval_metric="logloss",
    use_label_encoder=False,
    random_state=12,
)
# Ajustar el modelo
xgb_model.fit(train_X, train_y)

xgb_train_time = time.time() - start_time # Tiempo de entrenamiento

--Evaluación del modelo--

In [ ]:
# Evaluar el modelo
xgb_preds = xgb_model.predict(test_X)
xgb_accuracy = accuracy_score(test_y, xgb_preds) # Precisión
xgb_f1 = f1_score(test_y, xgb_preds) # F1 Score

In [ ]:
# Imprimir resultados
print("Resultados modelo XGBoost")
print(f"Tiempo de entrenamiento: {xgb_train_time:.2f} segundos")
print(f"Precisión: {100 * xgb_accuracy:.2f}%")
print(f"F1 Score: {100 * xgb_f1:.2f}%")
print("Reporte de clasificación:\n", classification_report(test_y, xgb_preds))

--Importacias de caracteristicas--

In [ ]:
xgb_importances = pd.Series(xgb_model.feature_importances_, index=train_X.columns) # Importancias de características
xgb_importances = xgb_importances[xgb_importances > 0].sort_values() # Filtrar y ordenar importancias

In [ ]:
#Representación gráfica de Importancias
plt.figure(figsize=(8, 10))
xgb_importances.tail(20).plot(kind='barh') # Top 20
plt.title("Importancias de Características - XGBoost")
plt.xlabel("Importancia")
plt.show()

--Entrenamiento del modelo LigthGBM--

In [ ]:
start_time = time.time() # Iniciar temporizador
# Entrenar el modelo
lgb_model = lgb.LGBMClassifier(
    objective='binary', # Objetivo de clasificación binaria
    metric='binary_logloss', # Métrica de evaluación
    random_state=12, # Semilla para la aleatoriedad
)
# Ajustar el modelo
lgb_model.fit(train_X, train_y)

lgb_train_time = time.time() - start_time # Tiempo de entrenamiento

--Evaluación del modelo--

In [ ]:
# Evaluar el modelo
lgb_preds = lgb_model.predict(test_X)
lgb_accuracy = accuracy_score(test_y, lgb_preds) # Precisión
lgb_f1 = f1_score(test_y, lgb_preds) # F1 Score

In [ ]:
# Imprimir resultados
print("Resultados modelo LightGBM")
print(f"Tiempo de entrenamiento: {lgb_train_time:.2f} segundos")
print(f"Precisión: {100 * lgb_accuracy:.2f}%")
print(f"F1 Score: {100 * lgb_f1:.2f}%")
print("Reporte de clasificación:\n", classification_report(test_y, lgb_preds))

--Importacias de caracteristicas--

In [ ]:
lgb_importances = pd.Series(lgb_model.feature_importances_, index=train_X.columns) # Importancias de características
lgb_importances = lgb_importances[lgb_importances > 0].sort_values() # Filtrar y ordenar importancias

In [ ]:
#Representación gráfica de Importancias
plt.figure(figsize=(8, 10))
lgb_importances.tail(20).plot(kind='barh') # Top 20
plt.title("Importancias de Características - LightGBM")
plt.xlabel("Importancia")
plt.show()

--Entrenamiento del modelo CatBoost--

In [ ]:
start_time = time.time() # Iniciar temporizador
# Entrenar el modelo
cat_model = CatBoostClassifier(
    iterations=1000, # Número de iteraciones
    learning_rate=0.1, # Tasa de aprendizaje
    depth=6, # Profundidad del árbol
    loss_function='Logloss', # Función de pérdida
    random_seed=12, # Semilla para la aleatoriedad
    verbose=0
)
# Ajustar el modelo
cat_model.fit(train_X, train_y)

cat_train_time = time.time() - start_time # Tiempo de entrenamiento

--Evaluación del modelo--

In [ ]:
# Evaluar el modelo
cat_preds = cat_model.predict(test_X)
cat_accuracy = accuracy_score(test_y, cat_preds) # Precisión
cat_f1 = f1_score(test_y, cat_preds) # F1 Score

In [ ]:
# Imprimir resultados
print("Resultados modelo CatBoost")
print(f"Tiempo de entrenamiento: {cat_train_time:.2f} segundos")
print(f"Precisión: {100 * cat_accuracy:.2f}%")
print(f"F1 Score: {100 * cat_f1:.2f}%")
print("Reporte de clasificación:\n", classification_report(test_y, cat_preds))

--Importacias de caracteristicas--

In [ ]:
cat_importances = pd.Series(cat_model.feature_importances_, index=train_X.columns) # Importancias de características
cat_importances = cat_importances[cat_importances > 0].sort_values() # Filtrar y ordenar importancias

In [ ]:
#Representación gráfica de Importancias
plt.figure(figsize=(8, 10))
cat_importances.tail(20).plot(kind='barh') # Top 20
plt.title("Importancias de Características - CatBoost")
plt.xlabel("Importancia")
plt.show()

--Comparativa de modelos--

In [ ]:
# Guardar resultados de XGBoost
results = []
results.append({
    "Modelo": "XGBoost",
    "Precisión": xgb_accuracy * 100,
    "F1-score": xgb_f1 * 100,
    "Tiempo (s)": xgb_train_time
})

# Guardar resultados de LightGBM
results.append({
    "Modelo": "LightGBM",
    "Precisión": lgb_accuracy * 100,
    "F1-score": lgb_f1 * 100,
    "Tiempo (s)": lgb_train_time
})

# Guardar resultados de CatBoost
results.append({
    "Modelo": "CatBoost",
    "Precisión": cat_accuracy * 100,
    "F1-score": cat_f1 * 100,
    "Tiempo (s)": cat_train_time
})

In [ ]:
# Crear DataFrame con los resultados
df_results = pd.DataFrame(results)

#Redondear resultados
df_results["Precisión"] = df_results["Precisión"].round(2).astype(str) + "%"
df_results["F1-score"] = df_results["F1-score"].round(2).astype(str) + "%"
df_results["Tiempo (s)"] = df_results["Tiempo (s)"].round(2).astype(str) + " s"

print("\nResultados Comparativos:")
print(df_results.sort_values(by="Precisión", ascending=False))
